# Multiple Speer Blvds

Are there two Speer Boulevards in the routes? How to handle them? 

In [1]:
import json
import pytz
import folium
import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

cda = CrashDataAnalysis()

In [6]:
query = f"""
with route_centerline_count as (
    select
    sr.gid
    , sr.lrsroute
    , sc.fullname
    , count(distinct sc.masterid) as num_centerlines
    , row_number() over (partition by sr.gid, sr.lrsroute order by count(distinct sc.masterid) desc) 
        as fullname_priority

    from street_routes sr
    inner join street_centerline sc using (lrsroute)

    group by 1,2,3
)

select
sr.gid
, sr.lrsroute
, rcc.fullname
, st_AsGeoJSON(sr.geom) as street_line
, st_length(sr.geom_denver) / 5280 as length_miles

from street_routes sr
inner join route_centerline_count rcc on (rcc.gid = sr.gid and rcc.fullname_priority = 1)

-- where sr.gid in (3900, 3901)
where sr.gid in (1452, 1453)
"""

geom = pd.read_sql(query, cda.conn)
geom

,gid,lrsroute,fullname,street_line,length_miles
0,1452,ESPEERN BD,E SPEER BLVD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",1.060124
1,1453,ESPEERS BD,E SPEER BLVD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",0.976285


In [7]:
this_map = folium.Map(prefer_canvas=True)
folium.GeoJson(geom.iloc[0]['street_line'], style_function=lambda x: {'color': 'red'}).add_to(this_map)
folium.GeoJson(geom.iloc[1]['street_line'], style_function=lambda x: {'color': 'blue'}).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map